In [32]:
import pandas as pd
import numpy as np

pd.set_option('max_rows', 20)

DATA_FILE_PATH_WIN = 'D:/analytics/stock/hist'
TEST_DATA_FILE_PATH = 'D:/analytics/stock/hist/002415.csv'

df = pd.read_csv(TEST_DATA_FILE_PATH, index_col=0, parse_dates=True, \
                 usecols=['date', 'close', 'p_change', 'ma5', 'ma10', 'ma20'], \
                 error_bad_lines=False)
df

,close,p_change,ma5,ma10,ma20
date,,,,,
2016-04-22,31.17,2.37,30.686,30.857,30.809
2016-04-21,30.45,0.79,30.732,30.806,30.803
2016-04-20,30.21,-1.60,30.880,30.831,30.875
2016-04-19,30.70,-0.65,31.064,30.946,30.953
2016-04-18,30.90,-1.59,31.004,31.016,31.038
2016-04-15,31.40,0.67,31.028,30.976,31.037
2016-04-14,31.19,0.19,30.880,30.918,30.967
2016-04-13,31.13,2.40,30.782,30.909,30.868
2016-04-12,30.40,-2.00,30.828,30.781,30.792


In [5]:
r = df.resample('W')
# daily average price change percent on weekly basis
r['p_change'].mean().dropna().sort_values()

date
2015-07-12   -5.413333
2015-06-21   -3.862000
2013-12-08   -2.440000
2015-03-08   -2.382500
2014-03-23   -2.336000
                ...   
2015-04-05    2.442000
2013-06-16    2.645000
2014-12-07    2.842000
2015-01-11    3.418000
2015-05-24    6.086000
Name: p_change, dtype: float64

In [33]:
WEEKLY_DATA_FOLDER = 'D:/analytics/stock/hist-W-2016-0627'
WEEKLY_SAMPLE_PATH = 'D:/analytics/stock/hist-W-2016-0627/002643.csv'

df = pd.read_csv(WEEKLY_SAMPLE_PATH, index_col=0, parse_dates=True, \
                 usecols=['date', 'close', 'p_change', 'ma5', 'ma10', 'ma20'], \
                 error_bad_lines=False)
df

,close,p_change,ma5,ma10,ma20
date,,,,,
2016-06-27,58.10,3.11,52.744,50.356,42.310
2016-06-24,56.35,7.33,50.660,48.755,41.205
2016-06-17,52.50,6.56,49.130,47.041,40.087
2016-06-08,49.27,3.73,48.250,45.011,39.300
2016-06-03,47.50,-0.38,47.926,43.202,38.561
2016-05-27,47.68,-2.09,47.968,41.792,38.409
2016-05-20,48.70,1.25,46.850,40.089,38.051
2016-05-13,48.10,0.94,44.952,38.317,37.742
2016-05-06,47.65,-0.13,41.772,36.678,37.301


In [12]:
r_df = df.sort_index()
r_df.rolling(window=5)['close'].mean()

date
2016-01-08       NaN
2016-01-15       NaN
2016-01-22       NaN
2016-01-29       NaN
2016-02-05    37.140
               ...  
2016-06-03    47.926
2016-06-08    48.250
2016-06-17    49.130
2016-06-24    50.660
2016-06-27    52.744
Name: close, dtype: float64

In [30]:
pct_change = df['ma5'].pct_change().fillna(0)  # trim off first N/A element
pct_change

date
2016-06-27    0.000000
2016-06-24   -0.039512
2016-06-17   -0.030201
2016-06-08   -0.017912
2016-06-03   -0.006715
                ...   
2016-02-05    0.038242
2016-01-29    0.024394
2016-01-22    0.044735
2016-01-15    0.012730
2016-01-08    0.015402
Name: ma5, dtype: float64

In [36]:
s = df['ma5']
change_point = None
for i in range(0, len(s)-1):
    if s[i] < s[i+1]:
        change_point = i
        break
if not change_point:
    change_point = len(s) - 1
print(df.index[change_point])

2016-06-03 00:00:00
date
2016-06-27          NaN
2016-06-24          NaN
2016-06-17          NaN
2016-06-08          NaN
2016-06-03    18.244406
Name: close, dtype: float64
